In [1]:
import pandas as pd
import numpy as np
import datetime
import sklearn
import tsfresh
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute
import warnings
warnings.filterwarnings('ignore')
import datetime
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import *
from sklearn.neural_network import MLPRegressor
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
data = pd.read_csv('energydata_Transformation.csv', parse_dates=['date'])

In [4]:
weekType = pd.get_dummies(data['weekType'], prefix = 'weekType')
day_of_week = pd.get_dummies(data['day_of_week'], prefix = 'day_of_week')
#['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
#['Weekend', 'Weekday']

# Concat above dummies variable dataframe to the main dataframe
data = pd.concat((data,weekType),axis=1)
data = pd.concat((data,day_of_week),axis=1)

# Drop the WeekStatus and Day_of_week column
data = data.drop(['weekType','day_of_week', 'time'],axis=1)

In [5]:
check_outliers = ['Appliances', 'temp_kitchen', 'hum_kitchen', 'temp_living',
       'hum_living', 'temp_laundry', 'hum_laundry', 'temp_office',
       'hum_office', 'temp_bathroom', 'hum_bathroom', 'temp_building_out',
       'hum_building_out', 'temp_ironing', 'hum_ironing', 'temp_teenRoom',
       'hum_teenRoom', 'hum_parentRoom', 'Pressure', 'hum_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1']

for x in data[check_outliers]:
    data = data[np.abs(data[x]-data[x].mean()) <= (3*data[x].std())]

In [10]:
data_train,data_test = train_test_split(data,train_size=0.7,random_state=42)
column_list =['temp_kitchen', 'hum_kitchen', 'temp_living',
       'hum_living', 'temp_laundry', 'hum_laundry', 'temp_office',
       'hum_office', 'temp_bathroom', 'hum_bathroom', 'temp_building_out',
       'hum_building_out', 'temp_ironing', 'hum_ironing', 'temp_teenRoom',
       'hum_teenRoom', 'hum_parentRoom', 'Pressure','Visibility', 'month']
x_train=data_train.iloc[:,1:]
x_train= x_train[column_list]
print(x_train.shape)
y_train=data_train['Appliances']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=data_test.iloc[:,1:]
x_test = x_test[column_list]
print(x_test.shape)
y_test=data_test['Appliances']
x_test_sc=scaler.transform(x_test)

(12413, 20)
(5321, 20)


In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [12]:
# Xtrn, Xtest, Ytrn, Ytest = train_test_split(features_filtered,y,test_size=0.3,random_state=42)

# model = RandomForestRegressor(n_estimators=150, max_features='sqrt', n_jobs=-1)  # случайный лес
models = [RandomForestRegressor()]

TestModels = pd.DataFrame()
tmp = {}

for model in models:
    # get model name
    m = str(model)
    tmp['Model'] = m[:m.index('(')]
    # fit model on training dataset
    model.fit(x_train_sc, y_train)
    # predict consumption
    predictions = model.predict(x_test)
    #Evaluation for Testing set
    #R2 score
    tmp['R2_Test'] = r2_score(y_test,predictions)
    #Mean Absolute Error(MAE)
    tmp['MAE_Test']= mean_absolute_error(y_test,predictions)
    #Mean Squared Error(MSE)
    tmp['MSE_Test']= mean_squared_error(y_test,predictions)
    #Root Mean Squared Error (RMSE)
    tmp['RMSE_Test'] = np.sqrt(mean_squared_error(y_test,predictions))
    #Evaluation for Training test
    predictions_trn = model.predict(x_train_sc)
    #R2_Score
    tmp['R2_Train'] = r2_score(y_train,predictions_trn)
    #Mean Absolute Error(MAE)
    tmp['MAE_Train']= mean_absolute_error(y_train,predictions_trn)
    #Mean Squared Error(MSE)
    tmp['MSE_Train']= mean_squared_error(y_train,predictions_trn)
    #Root Mean Squared Error (RMSE)
    tmp['RMSE_Train'] = np.sqrt(mean_squared_error(y_train,predictions_trn))
    
    tmp['Training Score(%)'] = round(model.score(x_train_sc, y_train) * 100,3)
    tmp['Testing Score(%)'] = round(model.score(x_test, y_test) * 100,3)
    
    tmp['MAPE_Test'] =  round(mean_absolute_percentage_error(y_test,predictions))
    tmp['MAPE_Train'] =  round(mean_absolute_percentage_error(y_train,predictions_trn))
    # write obtained data
    TestModels = TestModels.append([tmp])

TestModels.set_index('Model', inplace=True)

In [13]:
TestModels

,MAE_Test,MAE_Train,MAPE_Test,MAPE_Train,MSE_Test,MSE_Train,R2_Test,R2_Train,RMSE_Test,RMSE_Train,Testing Score(%),Training Score(%)
Model,,,,,,,,,,,,
RandomForestRegressor,113.471904,9.910658,202.0,11.0,14120.143582,424.16958,-1.696863,0.909329,118.82821,20.595378,-169.686,90.933


Cross Validation

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Initialize the model based on best performance from above, We got ExtraTreesRegressor 
sel_model = RandomForestRegressor(random_state=42)

# Define the parameter subset

param_grid = {
    "n_estimators": [10, 50, 100, 200, 250, 300, 500, 800],
    "max_features": ["auto", "sqrt", "log2"],
    "max_depth": [None, 10, 50, 100, 200, 500]
}

# Use Randomized search to try 20 subsets from parameter space with 5-fold cross validation
random_search = RandomizedSearchCV(sel_model, param_grid, n_iter=20, scoring="r2", cv=5, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(x_train_sc, y_train)

In [16]:
# Best Parameters for the model from Randomized Search CV
print(random_search.best_params_)

{'n_estimators': 500, 'max_features': 'log2', 'max_depth': 100}


In [28]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mae = mean_absolute_error(test_labels,predictions)
    mape = 100 * np.mean(errors / test_labels)
    rmse = np.sqrt(mean_squared_error(test_labels,predictions))
    accuracy = model.score(test_features, test_labels)
    print('Model Performance')
    print('R2 : {:0.3f}'.format(accuracy))
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('RMSE : {:0.4f}'.format(rmse)) 
    print('MAPE : {:0.4f}'.format(mae*100))
    print('MAE: {:0.4f}'.format(mae))
    return accuracy

In [19]:
sel_model.get_params

<bound method BaseEstimator.get_params of RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)>

In [22]:
base_model = RandomForestRegressor(random_state = 42)
base_model.fit(x_train_sc, y_train)
base_accuracy = evaluate(base_model, x_test, y_test)

Model Performance
R2 : -4.339
Average Error: 160.6593 degrees.
RMSE : 167.1899
MAPE : 16065.9275
MAE: 160.6593


In [29]:
base_accuracy = evaluate(base_model, x_train_sc, y_train)

Model Performance
R2 : 0.913
Average Error: 9.8063 degrees.
RMSE : 20.1272
MAPE : 980.6252
MAE: 9.8063


In [30]:
best_model = random_search.best_estimator_
best_accuracy = evaluate(best_model, x_train_sc, y_train)

Model Performance
R2 : 0.941
Average Error: 8.3618 degrees.
RMSE : 16.5800
MAPE : 836.1787
MAE: 8.3618


In [32]:
from sklearn.cross_validation import cross_val_score

sel_model = RandomForestRegressor()
scores = cross_val_score(sel_model, x_test, y_test, cv=5)
scores.mean()

0.3244846837349832

Regularization

In [43]:
from sklearn.linear_model import Ridge

## training the model

ridgeReg = Ridge(alpha=0.05, normalize=True)

ridgeReg.fit(x_train_sc,y_train)

predictions = ridgeReg.predict(x_train_sc)

print('R2 :{:.3f}'.format(ridgeReg.score(x_train_sc,y_train)))
print('MSE: {:.3f}'.format(mean_squared_error(y_train,predictions)))
print('RMSE: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,predictions))))
print('MAE: {:.3f}'.format(mean_absolute_error(y_train,predictions)))
print('MAPE:{:.3f}'.format(np.mean(np.abs((y_train - predictions) / y_train)) * 100))

R2 :0.165
MSE: 3907.911
RMSE: 62.513
MAE: 38.030
MAPE:50.354


In [44]:
from sklearn.linear_model import Lasso

## training the model

LassoReg = Lasso(alpha=0.005, normalize=True)

LassoReg.fit(x_train_sc,y_train)

predictions = LassoReg.predict(x_train_sc)
print('MSE: {:.3f}'.format(mean_squared_error(y_train,predictions)))
print('RMSE: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,predictions))))
print('MAE: {:.3f}'.format(mean_absolute_error(y_train,predictions)))
print('MAPE:{:.3f}'.format(np.mean(np.abs((y_train - predictions) / y_train)) * 100))

MSE: 3897.544
RMSE: 62.430
MAE: 37.954
MAPE:50.106


In [45]:
from sklearn.linear_model import ElasticNet

## training the model

ElasticNetReg = ElasticNet(alpha=0.001, normalize=True)

ElasticNetReg.fit(x_train_sc,y_train)

predictions = ElasticNetReg.predict(x_train_sc)
print('MSE: {:.3f}'.format(mean_squared_error(y_train,predictions)))
print('RMSE: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,predictions))))
print('MAE: {:.3f}'.format(mean_absolute_error(y_train,predictions)))
print('MAPE:{:.3f}'.format(np.mean(np.abs((y_train - predictions) / y_train)) * 100))

MSE: 4570.834
RMSE: 67.608
MAE: 43.543
MAPE:59.585
